<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso-data-management/blob/main/tutorials/02_Upload_new_footage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">Colab KSO Tutorials #2: Upload new footage</h1>
<h3 align="right">Written by the KSO team</h3>

# Set up KSO requirements

### Install all the requirement

In [ ]:
from IPython.display import clear_output
import os
import sys

try:
    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules -b master https://github.com/ocean-data-factory-sweden/kso.git
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/yolov5/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Navigate to the correct folder
    os.chdir("kso/tutorials")

except:
    IN_COLAB = False


# Ensure widgets are shown properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

clear_output()
if IN_COLAB == True:
    print("Running in Colab: All packages are installed and ready to go!")
else:
    print("Running locally... you're good to go!")

### Import Python packages

In [ ]:
# Set the directory of the libraries
try:
    if "kso_utils" not in sys.modules:
        sys.path.append("..")
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Import required modules
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
import kso_utils.server_utils as s_utils
from kso_utils.project import ProjectProcessor


print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

### Initiate project's database

In [ ]:
# Save the name of the project
project = p_utils.find_project(project_name=project_name.value)
# Initiate pp
pp = ProjectProcessor(project)

### Upload new movies to the server: Choose files to upload

In [ ]:
pp.add_movies()

# (Spyfish ONLY) Select the survey linked to the videos

In [ ]:
survey_i = kso_widgets.select_survey(pp.db_info)

In [ ]:
survey_name = kso_widgets.confirm_survey(survey_i, pp.db_info)

### Select new deployments

To save time you can select multiple deployments **recorded on the same day**

In [ ]:
deployment_selected, survey_row, survey_server_name = kso_widgets.select_deployment(
    project=project, db_info_dict=pp.db_info, survey_i=survey_i
)

### Specify the date of the deployments

In [ ]:
deployment_date = kso_widgets.select_eventdate(survey_row)

### Check the database to avoid deployment duplicates

In [ ]:
deployment_names = kso_widgets.check_deployment(
    deployment_selected, deployment_date, survey_server_name, pp.db_info, survey_i
)

### Update new deployment files

In [ ]:
movie_files_server = kso_widgets.update_new_deployments(
    deployment_filenames=deployment_names,
    db_info_dict=pp.db_info,
    survey_server_name=survey_server_name,
    deployment_date=deployment_date,
)

### Specify deployment details

In [ ]:
deployment_info = kso_widgets.record_deployment_info(deployment_names, pp.db_info)

### Review deployment  details

In [ ]:
new_deployment_row = kso_widgets.confirm_deployment_details(
    deployment_names,
    survey_server_name,
    pp.db_info,
    survey_i,
    deployment_info,
    movie_files_server,
    deployment_date,
)

# !!!Only pass this point if deployment details are correct!!!

### Update movies csv and upload video to s3

In [ ]:
s_utils.upload_concat_movie(pp.db_info, new_deployment_row)

In [ ]:
# END